In [1]:
import numpy as np
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization
from keras.models import Sequential
from keras.losses import MeanSquaredError, BinaryCrossentropy
from keras.regularizers import l1
from keras.datasets import boston_housing
# from keras.utils import normalize

Using TensorFlow backend.


In [2]:
def standardize(X, eps=1e-10):
    m = X.mean(axis=0, keepdims=True)
    s = X.std(axis=0, keepdims=True)
    s[np.where(s == 0)] = eps
    return (X - m) / s


def normalize(X, eps=1e-10):
    mn = X.min(axis=0, keepdims=True)
    mx = X.max(axis=0, keepdims=True)
    diff = mx - mn
    diff[np.where(diff == 0)] = eps
    return (X - mn) / (mx - mn)

In [3]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()
X_train = normalize(X_train)
X_test = normalize(X_test)
y_train = y_train[:, np.newaxis]
y_test = y_test[:, np.newaxis]

d = X_train.shape[1]
# enc_d = 32
enc_d = d

In [4]:
model = Sequential()
model.add(Dense(8, input_dim=d, activation="relu"))
model.add(BatchNormalization())
model.add(Dense(4, activation="relu"))
model.add(Dense(1))
model.compile(optimizer="adam", loss=MeanSquaredError())
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Train on 404 samples, validate on 102 samples
Epoch 1/50
404/404 [==============================] - 1s 1ms/step - loss: 597.9693 - val_loss: 630.7644
Epoch 2/50
404/404 [==============================] - 0s 104us/step - loss: 588.7445 - val_loss: 627.6167
Epoch 3/50
404/404 [==============================] - 0s 109us/step - loss: 580.5140 - val_loss: 624.9766
Epoch 4/50
404/404 [==============================] - 0s 109us/step - loss: 573.8225 - val_loss: 622.4166
Epoch 5/50
404/404 [==============================] - 0s 106us/step - loss: 567.2196 - val_loss: 620.0293
Epoch 6/50
404/404 [==============================] - 0s 114us/step - loss: 560.8313 - val_loss: 617.4960
Epoch 7/50
404/404 [==============================] - 0s 116us/step - loss: 554.2473 - val_loss: 614.3205
Epoch 8/50
404/404 [==============================] - 0s 114us/step - loss: 546.7198 - val_loss: 610.1485
Epoch 9/50
404/404 [==============================] - 0s 109us/step - loss: 538.6788 - val_loss: 604.4006
Ep

In [5]:
encoder = Sequential()
encoder.add(Dense(enc_d, input_dim=d, activation="relu", activity_regularizer=l1(10e-5)))

decoder = Sequential()
decoder.add(Dense(d, activation="sigmoid", input_dim=enc_d))

auto_enc = Sequential()
auto_enc.add(encoder)
auto_enc.add(decoder)
auto_enc.compile(optimizer="adadelta", loss=BinaryCrossentropy(), metrics=["accuracy"])
hist = auto_enc.fit(X_train, X_train, batch_size=32, epochs=50, validation_data=(X_test, X_test))

Train on 404 samples, validate on 102 samples
Epoch 1/50
404/404 [==============================] - 0s 354us/step - loss: 0.6993 - accuracy: 0.0912 - val_loss: 0.6950 - val_accuracy: 0.1071
Epoch 2/50
404/404 [==============================] - 0s 89us/step - loss: 0.6738 - accuracy: 0.1114 - val_loss: 0.6742 - val_accuracy: 0.1365
Epoch 3/50
404/404 [==============================] - 0s 89us/step - loss: 0.6517 - accuracy: 0.1386 - val_loss: 0.6551 - val_accuracy: 0.1440
Epoch 4/50
404/404 [==============================] - 0s 101us/step - loss: 0.6307 - accuracy: 0.1590 - val_loss: 0.6366 - val_accuracy: 0.1659
Epoch 5/50
404/404 [==============================] - 0s 87us/step - loss: 0.6097 - accuracy: 0.1750 - val_loss: 0.6183 - val_accuracy: 0.1780
Epoch 6/50
404/404 [==============================] - 0s 84us/step - loss: 0.5891 - accuracy: 0.1797 - val_loss: 0.5977 - val_accuracy: 0.1795
Epoch 7/50
404/404 [==============================] - 0s 92us/step - loss: 0.5684 - accuracy: 

In [6]:
X_est = encoder.predict(X_test)

In [7]:
X_est[0]

array([0.        , 2.7750661 , 0.        , 0.12134586, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       1.8331033 , 2.2232275 , 0.        ], dtype=float32)